# Data Exploration

## Libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
sns.set()

In [ ]:
path = 'data/'
X_train = pd.read_csv( path + 'X_train.csv')
Y_train = pd.read_csv(path + 'Y_train.csv')
X_test = pd.read_csv(path + 'X_test.csv')

- ID: Unique row identifier, associated with a day (DAY_ID) and a country (COUNTRY),
- DAY_ID: Day identifier - dates have been anonymized, but all data corresponding to a specific day is consistent,
- COUNTRY: Country identifier - DE = Germany, FR = France, 

and then contains daily commodity price variations:

- GAS_RET: European gas,
- COAL_RET: European coal,
- CARBON_RET: Carbon emissions futures,

weather measures (daily, in the country x):

- x_TEMP: Temperature,
- x_RAIN: Rainfall,
- x_WIND: Wind,

energy production measures (daily, in the country x),

- x_GAS: Natural gas,
- x_COAL: Hard coal,
- x_HYDRO: Hydro reservoir,
- x_NUCLEAR: Daily nuclear production,
- x_SOLAR: Photovoltaic,
- x_WINDPOW: Wind power,
- x_LIGNITE: Lignite,

and electricity use metrics (daily, in the country x),

- x_CONSUMPTON: Total electricity consumption,
- x_RESIDUAL_LOAD: Electricity consumption after using all renewable energies,
- x_NET_IMPORT: Imported electricity from Europe,
- x_NET_EXPORT: Exported electricity to Europe,
- DE_FR_EXCHANGE: Total daily electricity exchange between Germany and France,
- FR_DE_EXCHANGE: Total daily electricity exchange between France and Germany.

Output data sets are composed of two columns:

- ID: Unique row identifier - corresponding to the input identifiers,
- TARGET: Daily price variation for futures of 24H electricity baseload.

In [ ]:
data = pd.merge(X_train, Y_train, on=['ID'])
data.head()

In [ ]:
data.describe()

In [ ]:
test = data.dropna()
sns.heatmap(test.corr(numeric_only=True),cmap='BrBG')

In [ ]:
all_data = data.copy()
all_data['train'] = 1
all_data = pd.concat([all_data,X_test])
all_data['TARGET'] = all_data['TARGET'].fillna(0)
all_data['train'] = all_data['train'].fillna(0)
all_data.describe()

In [ ]:
all_data.isnull().sum()

In [ ]:
print("the number of samples = ", len(all_data))
print("the number of samples of France = ", len(all_data[all_data['COUNTRY']=='FR']))
print("the number of samples of France = ", len(all_data[all_data['COUNTRY']=='DE']))

In [ ]:
# For one day, we report one price in each country
print(all_data[all_data['COUNTRY']=='FR']['DAY_ID'].value_counts().value_counts())
print(all_data[all_data['COUNTRY']=='DE']['DAY_ID'].value_counts().value_counts())

In [ ]:
# For some days we only have the electricity price in one country
all_data['DAY_ID'].value_counts().value_counts()

In [ ]:
day_id_count = all_data['DAY_ID'].value_counts()
day_id_count_1_index = np.array(day_id_count[day_id_count<2].index)
print(len(day_id_count_1_index))
all_data['COUNTRY'][all_data['DAY_ID'].isin(day_id_count_1_index)].value_counts()

In [ ]:
print(f"in 208 days,{(208/1494)*100}\% of the data, we have only the price in FR in training data")

For DE, we always have the price of DE and FR.

For FR, in 208 days we don't have the corresponding price in DE.

IDEAS: 

1- Get the missing values of other columns using same DAY_ID.

2- Create a model that is able to predict the price of a country based on the price of the other country.

## Get the missing values of columns using the columns of DAY_ID

In [ ]:
all_data_clean = all_data.copy() # it will represent the data without missing values
IDs = all_data_clean[['ID','COUNTRY','TARGET','DAY_ID']]
all_data_clean = all_data_clean.drop(['ID','COUNTRY','TARGET'],axis=1)
all_data_clean = all_data_clean.groupby(['DAY_ID']).mean().reset_index()
all_data_clean = pd.merge(left = all_data_clean, right = IDs, on=['DAY_ID'])

In [ ]:
all_data_clean 

In [ ]:
(all_data.isnull().sum() - all_data_clean.isnull().sum())

No improvement, we still have the same null values

In [ ]:
all_data.isnull().sum()

In [ ]:
all_data[all_data['FR_DE_EXCHANGE'].isnull()].isnull().sum()

If we don't know the exchange between FR and DE, we don't know the net import and export.
They represent 1% of the data.(Let us put them to zero)

In [ ]:
all_data_clean['DE_FR_EXCHANGE'] = all_data_clean['DE_FR_EXCHANGE'].fillna(0)
all_data_clean['FR_DE_EXCHANGE'] = all_data_clean['FR_DE_EXCHANGE'].fillna(0)

In [ ]:
all_data_clean[['DE_FR_EXCHANGE','FR_DE_EXCHANGE']] #They are colinear, we should keep only one of them

In [ ]:
all_data_clean.isnull().sum()

What is the relation between the net export and the exchange?

In [ ]:
test = all_data_clean.dropna()
plt.scatter(test['DE_NET_EXPORT'],test['DE_FR_EXCHANGE'])
print(np.corrcoef(test['DE_NET_EXPORT'],test['DE_FR_EXCHANGE']))
plt.show()

In [ ]:
test = all_data_clean.dropna()
plt.scatter(test['FR_NET_EXPORT'],test['FR_DE_EXCHANGE'])
print(np.corrcoef(test['FR_NET_EXPORT'],test['FR_DE_EXCHANGE']))
plt.show()

Let us regress the net export from the exchange

In [ ]:
def regress_var(df, x_columns, y_column, model, out = True):
    temp = df.dropna().copy()
    X = np.array(temp[x_columns])
    y = np.array(temp[y_column])
    model.fit(X,y)
    if(out):
        print("Model score", model.score(X,y))
    return df.apply(lambda row: model.predict(np.array(row[x_columns]).reshape(-1,len(x_columns)))[0] if(np.isnan(row[y_column])) else row[y_column], axis=1)


In [ ]:
x_columns = ['DE_FR_EXCHANGE']
y_column = 'DE_NET_EXPORT'
all_data_clean[y_column] = regress_var(all_data_clean, x_columns, y_column, LinearRegression(), out=True)
all_data_clean['DE_NET_IMPORT'] = - all_data_clean['DE_NET_EXPORT']
all_data_clean

In [ ]:
x_columns = ['FR_DE_EXCHANGE']
y_column = 'FR_NET_EXPORT'
all_data_clean[y_column] = regress_var(all_data_clean, x_columns, y_column, LinearRegression(), out=True)
all_data_clean['FR_NET_IMPORT'] = - all_data_clean['FR_NET_EXPORT']
all_data_clean

In [ ]:
all_data_clean.isnull().sum()

In [ ]:
all_data_clean = all_data_clean.drop(['DE_FR_EXCHANGE', 'FR_NET_IMPORT','DE_NET_IMPORT'],axis=1)
all_data_clean['FR_NET_EXPORT'] -= all_data_clean['FR_DE_EXCHANGE']
all_data_clean['DE_NET_EXPORT'] += all_data_clean['FR_DE_EXCHANGE']

Replace all the other by the mean for the moment

In [ ]:
all_data_clean = all_data_clean.fillna(all_data_clean.mean(numeric_only=True))
all_data_clean

In [ ]:
all_data_clean.isnull().sum()

In [ ]:
all_data_clean['COUNTRY'] = all_data_clean['COUNTRY'].apply(lambda x: 0 if(x == 'FR') else 1)

In [ ]:
all_data_clean.var() # It seems that they are already normalized

In [115]:
X_train = all_data_clean[all_data_clean['train'] == 1].drop(['train','TARGET','ID','DAY_ID','COUNTRY'],axis=1)
X_test = all_data_clean[all_data_clean['train'] == 0].drop(['train','TARGET','DAY_ID','COUNTRY'],axis=1)
id_test = X_test['ID']
X_test = X_test.drop('ID',axis=1)
y_train = all_data_clean[all_data_clean['train'] == 1]['TARGET']

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def pca_train_test(X_train, X_test, n_components = 5):
    pca = PCA(n_components =n_components)
    X_train_PCA = pca.fit_transform(X_train)
    X_test_PCA = pca.transform(X_test)
    return X_train_PCA,X_test_PCA

# Create the cross validation

In [266]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

def my_custom_loss_func_exp(y_true, y_pred):
    return spearmanr(np.exp(y_true), np.exp(y_pred)).correlation  

def my_custom_loss_func(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation

def evaluate_model(X, y, model, loss = my_custom_loss_func,  cv = 5):
    scores = cross_val_score(model, X, y, cv=cv, scoring = make_scorer(loss))
    print('scores = ', scores)
    print(scores.mean())
    #full train
    model.fit(X,y)
    train_score = loss(model.predict(X),y)
    print("Full train_score = ", train_score)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=40)
    model.fit(X_train,y_train)
    train_score = loss(model.predict(X_train),y_train)
    test_score = loss(model.predict(X_test),y_test)
    print("train_score = ", train_score)
    print("test_score = ",test_score)
    return [scores.mean(),test_score]
    

Benchmark

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV

valid_metric =['additive_chi2', 'chi2', 'linear', 'poly', 'polynomial', 'rbf',
        'laplacian', 'sigmoid', 'cosine']

In [ ]:
lasso_parameters = {
 'alpha': np.arange(0.00, 1.0, 0.005)
 }

In [ ]:
model = Lasso()
#evaluate_model(X_train, y_train, model)
clf = GridSearchCV(model, # model
     param_grid = lasso_parameters, # hyperparameters
     scoring= make_scorer(my_custom_loss_func), # metric for scoring
     cv=5,
     n_jobs=-1, error_score='raise', verbose=3)
clf.fit(X_train, y_train)

In [ ]:
clf.best_estimator_

In [ ]:
model = Lasso()
#evaluate_model(X_train, y_train, model)
clf = GridSearchCV(model, # model
     param_grid = lasso_parameters, # hyperparameters
     scoring= make_scorer(my_custom_loss_func_exp), # metric for scoring
     cv=5,
     n_jobs=-1, error_score='raise', verbose=3)
clf.fit(X_train, np.log(y_train+7))

In [ ]:
clf.best_estimator_

In [ ]:
evaluate_model(X_train, np.log(y_train+7), model, loss = my_custom_loss_func_exp)

In [ ]:
'''
for i in range(1,X_train.shape[1]):
    print(i)
    X_train_pca, X_test_pca = pca_train_test(X_train,X_test,n_components = i)
    evaluate_model(X_train_pca, y_train, model)
'''

In [ ]:
'''
for i in range(1,X_train.shape[1]):
    print(i)
    X_train_pca, X_test_pca = pca_train_test(X_train,X_test,n_components = i)
    evaluate_model(X_train_pca, np.log(y_train+10), model,loss = my_custom_loss_func_exp)
'''

It seems the projection on PCA very stable.

In [ ]:
X_train_pca, X_test_pca = pca_train_test(X_train,X_test,n_components = X_train.shape[1])
evaluate_model(X_train_pca, np.log(y_train+7), model,loss = my_custom_loss_func_exp)

Let us choose the best pca projection at each iteration

In [ ]:
best = []
acc = []
for i in range(1,X_train_pca.shape[1]):
    print(i)
    loss = my_custom_loss_func_exp
    clf = SequentialFeatureSelector(model, n_features_to_select=i, cv = 5 , scoring= make_scorer(loss))
    clf.fit(X_train_pca, np.log(y_train+7))
    best.append(clf.get_support())
    X_train_pca_filter = X_train_pca[:,clf.get_support()]
    acc.append(evaluate_model(X_train_pca_filter, np.log(y_train+7), model,loss = my_custom_loss_func_exp))

In [ ]:
best_features = best[11]
X_train_pca_filter = X_train_pca[:,best_features]
evaluate_model(X_train_pca_filter, np.log(y_train+7), model,loss = my_custom_loss_func_exp, cv = KFold(shuffle=True, n_splits=2))

# Submission

In [ ]:
y_benchmark = pd.read_csv("data/y_sub.csv")
y_benchmark

In [ ]:
lasso_parameters = {
 'alpha': np.arange(0.01, 0.015, 0.00001)
 }

In [ ]:
model = Lasso()
#evaluate_model(X_train, y_train, model)
clf = GridSearchCV(model, # model
     param_grid = lasso_parameters, # hyperparameters
     scoring= make_scorer(my_custom_loss_func), # metric for scoring
     cv=5,
     n_jobs=-1, error_score='raise', verbose=3)
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
model = Lasso(0.0152,)
evaluate_model(X_train, y_train, model)
model.fit(X_train,y_train)

y_sub = model.predict(X_test)
sub = pd.DataFrame()
sub['ID'] = id_test
sub['TARGET'] = y_sub
#sub.to_csv("submissions/4_sub.csv",index=False)

In [436]:
from scipy.optimize import minimize
import numpy as np

def mean_absolute_percentage_error(y_pred, y_true, sample_weights=None):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert len(y_true) == len(y_pred)
    
    if np.any(y_true==0):
        print("Found zeroes in y_true. MAPE undefined. Removing from set...")
        idx = np.where(y_true==0)
        y_true = np.delete(y_true, idx)
        y_pred = np.delete(y_pred, idx)
        if type(sample_weights) != type(None):
            sample_weights = np.array(sample_weights)
            sample_weights = np.delete(sample_weights, idx)
        
    if type(sample_weights) == type(None):
        return(np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    else:
        sample_weights = np.array(sample_weights)
        assert len(sample_weights) == len(y_true)
        return(100/sum(sample_weights)*np.dot(
                sample_weights, (np.abs((y_true - y_pred) / y_true))
        ))
 

    

class CustomLinearModel:
    """
    Linear model: Y = XB, fit by minimizing the provided loss_function
    with L2 regularization
    """
    def __init__(self, loss_function=mean_absolute_percentage_error, 
                 X=None, Y=None, sample_weights=None, beta_init=None, 
                 regularization=0.001):
        self.regularization = regularization
        self.beta = None
        self.loss_function = loss_function
        self.sample_weights = sample_weights
        self.beta_init = beta_init
        
        self.X = X
        self.Y = Y
            
    def clip_beta(self):
        self.beta[np.abs(self.beta) <= 1e-5] = 0
    
    def predict(self, X):
        prediction = X@self.beta[1:]
        return(prediction)

    def model_error(self):
        error = self.loss_function(
            self.predict(self.X), self.Y, sample_weights=self.sample_weights
        )
        return(error)
    
    def l2_regularized_loss(self, beta):
        self.beta = beta
        print(np.var(self.X@self.beta[1:]))
        return(self.model_error() + \
               sum(self.regularization*np.abs(np.array(self.beta))) - self.beta[0]*(np.var(self.X@self.beta[1:])-1))
    
    def fit(self, maxiter=100000):        
        # Initialize beta estimates (you may need to normalize
        # your data and choose smarter initialization values
        # depending on the shape of your loss function)
        if type(self.beta_init)==type(None):
            # set beta_init = 1 for every feature
            self.beta_init = np.array([1]*(self.X.shape[1]+1))
        else: 
            # Use provided initial values
            pass
            
        if self.beta!=None and all(self.beta_init == self.beta):
            print("Model already fit once; continuing fit with more itrations.")
            
        res = minimize(self.l2_regularized_loss, self.beta_init,
                       method='Powell', options={'maxiter': maxiter})
        self.beta = res.x
        self.beta_init = self.beta
        self.clip_beta()



In [437]:
def mean_square_error(y_pred,y_true, sample_weights=None):
    return np.mean((y_pred-y_true)**2)
def error_corr(y_pred,y_true, sample_weights = None):
    return  1 - spearmanr(y_true, y_pred).correlation
    #return np.mean((y_pred-y_true)**2) - np.mean(y_pred**2)

In [439]:
l1_lasso_model = CustomLinearModel(
    loss_function = error_corr,
    X=np.array(X_train_pca), Y=np.array(y_train), regularization=0.1
)
l1_lasso_model.fit()
print(l1_lasso_model.beta)
y_pred = l1_lasso_model.predict(X_train_pca)
print(my_custom_loss_func(y_pred,y_train))
print(np.var(y_pred),np.var(y_train))

26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654856214
26.391481654

C:\Users\aliha\AppData\Local\Temp\ipykernel_10724\840514040.py:64: RuntimeWarning: overflow encountered in double_scalars
  sum(self.regularization*np.abs(np.array(self.beta))) - self.beta[0]*(np.var(self.X@self.beta[1:])-1))


224.84914695426735
225.4573260471746
225.83804321359645
226.12367763528655
226.12367763528655
230.2804563704876
242.87573152379764
230.2804563704876
234.23507972340022
237.20842079990294
239.2482019635265
240.58607252578506
241.44241608156452
241.98293124385012
242.3212911174356
242.57514682802272
242.57514682802272
245.55417499397691
254.58079917339433
245.55417499397691
248.38832497032806
250.51922184460238
251.98106668309697
252.93987501552468
253.55358859878496
253.94095824127723
254.18344976123808
254.36537989514758
254.36537989514758
257.0540501947732
265.2008735831679
257.0540501947732
259.61196319243675
261.535167277213
262.8545300294153
263.719885891402
264.2737824616481
264.6233962265156
264.8422527499554
265.00645064282514
265.00645064282514
267.5769202132169
275.3655886930007
267.5769202132169
270.0223809025098
271.8610360839424
273.122396321126
273.9497089563371
274.4792548360087
274.81349869778194
275.0227337363294
275.17971307559725
275.17971307559725
277.6271087813857
2

In [388]:
model = Lasso(0.014)
model.fit(X_train,y_train)
print(model.coef_)
y_pred = model.predict(X_train)
print(my_custom_loss_func(y_pred,y_train))

[-0.          0.          0.06113028 -0.2455976  -0.          0.
  0.         -0.         -0.          0.03193995  0.01413809  0.
  0.04386076 -0.01187838 -0.03324643 -0.02617754 -0.03482005 -0.02164598
  0.          0.         -0.         -0.00405012 -0.00943877  0.
 -0.01014029 -0.          0.01365529 -0.          0.00168504]
0.26809119247227353
